In [1]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_pendulum import get_pendulum_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf
tf.compat.v1.disable_eager_execution()


2024-06-06 08:13:52.796681: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 08:13:54.506394: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Generate data

In [2]:
training_data = get_pendulum_data(100, True)
validation_data = get_pendulum_data(10)

Generating pendulum data with noise
Generating pendulum data


# Set up model and training parameters

In [3]:
params = {}

params['input_dim'] = training_data['x'].shape[-1]
params['latent_dim'] = 1
params['model_order'] = 2
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(2*params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 5e-4
params['loss_weight_sindy_z'] = 5e-5
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [128,64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1000
params['learning_rate'] = 1e-4

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [4]:
num_experiments = 10
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'pendulum_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING


2024-06-06 08:14:15.948675: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-06 08:14:17.984520: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-06 08:14:17.984610: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-06 08:14:17.996628: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-06 08:14:17.996759: I external/local_xla/xla/stream_executor

Epoch 0
   training loss 0.015253981575369835, (0.007933256, 2.2651649, 14.395019, 0.995796)
   validation loss 0.01606769487261772, (0.008060615, 2.28555, 15.765689, 0.995796)
decoder loss ratio: 0.960995, decoder SINDy loss  ratio: 1.000000
Epoch 100
   training loss 0.01449159998446703, (0.0072873435, 2.5910507e-05, 14.395016, 0.67477417)
   validation loss 0.015283242799341679, (0.007393649, 3.58038e-05, 15.765689, 0.67477417)
decoder loss ratio: 0.881479, decoder SINDy loss  ratio: 1.000000
Epoch 200
   training loss 0.014473062008619308, (0.007272811, 0.0015033814, 14.395016, 0.26670074)
   validation loss 0.015279869548976421, (0.007394265, 0.0018520233, 15.765688, 0.26670074)
decoder loss ratio: 0.881552, decoder SINDy loss  ratio: 1.000000
Epoch 300
   training loss 0.014462979510426521, (0.007263424, 0.005699739, 14.395016, 0.17617254)
   validation loss 0.01527830958366394, (0.0073933285, 0.00749548, 15.765688, 0.17617254)
decoder loss ratio: 0.881441, decoder SINDy loss  ra

AttributeError: 'DataFrame' object has no attribute 'append'